In [1]:
!pip install ultralytics

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/Othercomputers/Desktop/SatellieteIMGDetection
import os
print(os.getcwd())



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Othercomputers/Desktop/SatellieteIMGDetection
/content/drive/Othercomputers/Desktop/SatellieteIMGDetection


In [ ]:

# 직접 YOLO 클래스를 임포트
from ultralytics import YOLO

# YOLO 모델 초기화
model = YOLO("yolo11x-obb.pt")
results = model.train(
    data="data_combine_colab.yaml",
    epochs=200,
    batch=2,
    imgsz=640,
    patience=100,
    device="0",
    lr0 = 0.001,
    # lrf = 0.1,
    scale = 0.022,
    # single_cls = True,
    augment = True,
    optimizer = 'AdamW',
    seed = 44,
)



Ultralytics 8.3.9 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=obb, mode=train, model=yolo11x-obb.pt, data=data_combine_colab.yaml, epochs=200, time=None, patience=100, batch=2, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=44, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=Tru

train: Scanning /content/drive/Othercomputers/Desktop/SatellieteIMGDetection/datasets/labels/train... 757 images, 0 backgrounds, 0 corrupt: 100%|██████████| 757/757 [07:53<00:00,  1.60it/s]


train: New cache created: /content/drive/Othercomputers/Desktop/SatellieteIMGDetection/datasets/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
val: Scanning /content/drive/Othercomputers/Desktop/SatellieteIMGDetection/datasets/labels/val... 92 images, 0 backgrounds, 0 corrupt: 100%|██████████| 92/92 [01:17<00:00,  1.19it/s]

val: New cache created: /content/drive/Othercomputers/Desktop/SatellieteIMGDetection/datasets/labels/val.cache


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      7/200      4.03G      1.444       1.41      1.348          2        640: 100%|██████████| 379/379 [01:57<00:00,  3.24it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 46/46 [00:08<00:00,  5.61it/s]

                   all         92       4910      0.661      0.518      0.573      0.367



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      8/200      3.81G      1.398      1.216       1.27         15        640:  99%|█████████▉| 377/379 [01:58<00:00,  2.84it/s]